# Notebook 1 – Data Cleaning, Feature Engineering, & Entity Resolution
**Project:** Judicial Vacancy → Nomination/Confirmation Pipeline

*Initial draft generated via ChatGPT model o3 on 2025-07-12T02:40:38.399372Z*

In [ ]:

import sys
from pathlib import Path

import pandas as pd
from loguru import logger
from rapidfuzz import fuzz, process

# Add the project root to the path so we can import our modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))


# Setup logging
logger.remove()  # Remove default handler
logger.add(sys.stderr, format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{function}</cyan> - <level>{message}</level>", level="INFO")

1

## Load dataframes from Raw data folder

Start with loading simpler, non-JSON-containing CSV files

In [ ]:
from nomination_predictor.config import INTERIM_DATA_DIR, RAW_DATA_DIR

# load FJC dataframes (and derived seat timeline)
fjc_judges = pd.read_csv(RAW_DATA_DIR / "judges.csv")
fjc_federal_judicial_service = pd.read_csv(RAW_DATA_DIR / "federal_judicial_service.csv")
fjc_demographics = pd.read_csv(RAW_DATA_DIR / "demographics.csv")
fjc_education = pd.read_csv(RAW_DATA_DIR / "education.csv")
fjc_other_federal_judicial_service = pd.read_csv(
    RAW_DATA_DIR / "other_federal_judicial_service.csv"
)
fjc_other_nominations_recess = pd.read_csv(RAW_DATA_DIR / "other_nominations_recess.csv")
fjc_professional_career = pd.read_csv(RAW_DATA_DIR / "professional_career.csv")
#seat_timeline = pd.read_csv(RAW_DATA_DIR / "seat_timeline.csv")

2025-07-14 17:57:11.737 | INFO     | nomination_predictor.config:<module>:103 - Project root: /home/wsl2ubuntuuser/nomination_predictor
2025-07-14 17:57:11.739 | INFO     | nomination_predictor.config:<module>:127 - Configuration loaded


Flatten JSON-containing congress DataFrames into separate DataFrames

In [ ]:
from nomination_predictor.features import flatten_json_dataframe

# Load Congress API dataframes
cong_nominations_raw = pd.read_csv(RAW_DATA_DIR / "nominations.csv")
cong_nominees_raw = pd.read_csv(RAW_DATA_DIR / "nominees.csv")

cong_nominations = flatten_json_dataframe(
    df=cong_nominations_raw,
    json_col="nomination",  # column containing the JSON data
    max_list_index=10,      # maximum number of list items to extract
    separator="_"           # separator for nested keys
)

cong_nominees= flatten_json_dataframe(
    df=cong_nominees_raw,
    json_col="nominee",
    max_list_index=5
)

2025-07-14 17:57:12.047 | INFO     | nomination_predictor.features:flatten_json_dataframe:316 - Flattening JSON data from column 'nomination' in 5746 rows
2025-07-14 17:57:14.242 | INFO     | nomination_predictor.features:flatten_json_dataframe:342 - Flattening complete. Original columns: 4, New columns: 37
2025-07-14 17:57:14.243 | INFO     | nomination_predictor.features:flatten_json_dataframe:316 - Flattening JSON data from column 'nominee' in 5671 rows
2025-07-14 17:57:15.782 | INFO     | nomination_predictor.features:flatten_json_dataframe:342 - Flattening complete. Original columns: 3, New columns: 34


In [ ]:
# Combine all dataframes into a single dictionary for bulk operations
# Start with FJC dataframes
dfs = {
    # FJC dataframes
    "fjc_judges": fjc_judges,
    "fjc_federal_judicial_service": fjc_federal_judicial_service,
    "fjc_demographics": fjc_demographics,
    "fjc_education": fjc_education,
    "fjc_other_federal_judicial_service": fjc_other_federal_judicial_service,
    "fjc_other_nominations_recess": fjc_other_nominations_recess,
    "fjc_professional_career": fjc_professional_career,
    #"seat_timeline": seat_timeline,
    
    # Congress dataframes
    "cong_nominations": cong_nominations,
    "cong_nominees": cong_nominees,
}

In [ ]:
# Print summary of available dataframes
print("Available dataframes:")
for name, df in dfs.items():
    print(f"- {name}: {len(df)} rows × {len(df.columns)} columns")

Available dataframes:
- fjc_judges: 4022 rows × 201 columns
- fjc_federal_judicial_service: 4720 rows × 30 columns
- fjc_demographics: 4022 rows × 18 columns
- fjc_education: 8040 rows × 6 columns
- fjc_other_federal_judicial_service: 611 rows × 31 columns
- fjc_other_nominations_recess: 828 rows × 4 columns
- fjc_professional_career: 19003 rows × 4 columns
- cong_nominations: 5746 rows × 37 columns
- cong_nominees: 5671 rows × 34 columns


#### Quick peek at all loaded dataframes

In [ ]:
logger.info("Checking for general shape and first handfuls of rows")
for name, df in dfs.items():
    print(f"{name:<35} → {df.shape}")
    print(df.head())  

2025-07-14 17:57:15.824 | INFO     | __main__:<module>:1 - Checking for general shape and first handfuls of rows
fjc_judges                          → (4022, 201)
        nid       jid last_name first_name middle_name suffix  birth_month  \
0  13761857  13761857   Abelson       Adam         Ben    NaN          NaN   
1   1393931      3419    Abrams     Ronnie                             NaN   
2   1376976         1   Abruzzo    Matthew          T.                 4.0   
3  13651551  13651551     Abudu      Nancy       Gbana    NaN          NaN   
4   1376981         2   Acheson     Marcus      Wilson                 6.0   

   birth_day birth_year  birth_city  ... degree_(3)  degree_year_(3)  \
0        NaN       1982   Cleveland  ...        NaN              NaN   
1        NaN       1968    New York  ...        NaN              NaN   
2       30.0       1889    Brooklyn  ...        NaN              NaN   
3        NaN       1974  Alexandria  ...        NaN              NaN   
4       

In [ ]:
logger.info("Checking for null values")
    
for name, df in dfs.items():
    print(df.isnull().sum())

2025-07-14 17:57:15.884 | INFO     | __main__:<module>:1 - Checking for null values
nid                                         0
jid                                         0
last_name                                   0
first_name                                  0
middle_name                                35
                                         ... 
school_(5)                               4017
degree_(5)                               4018
degree_year_(5)                          4017
professional_career                         4
other_nominations/recess_appointments    3307
Length: 201, dtype: int64
nid                                     0
sequence                                0
judge_name                              0
court_type                              0
court_name                              0
appointment_title                       0
appointing_president                    0
party_of_appointing_president          39
reappointing_president               4710
party_

## Data cleaning

#### Drop duplicated rows (if any) from FJC data

In [ ]:
for name, df in dfs.items():
    if name.startswith("fjc_") and name not in ("fjc_judges", "fjc_demographics", "fjc_other_nominations_recess"):
        dfs[name] = dfs[name].drop_duplicates(subset=["nid", "sequence"])
        print(f"shape of {name} after checking for & dropping duplicated nid sequenced items is { dfs[name].shape}")

shape of fjc_federal_judicial_service after checking for & dropping duplicated nid sequenced items is (4720, 30)
shape of fjc_education after checking for & dropping duplicated nid sequenced items is (8040, 6)
shape of fjc_other_federal_judicial_service after checking for & dropping duplicated nid sequenced items is (611, 31)
shape of fjc_professional_career after checking for & dropping duplicated nid sequenced items is (19003, 4)


### Normalize column names for DataFrames

In [ ]:
print("=== Column Names Before ===")

for name, df in dfs.items():
    print(f"{name:<35} → {df.columns.tolist()}")

=== Column Names Before ===
fjc_judges                          → ['nid', 'jid', 'last_name', 'first_name', 'middle_name', 'suffix', 'birth_month', 'birth_day', 'birth_year', 'birth_city', 'birth_state', 'death_month', 'death_day', 'death_year', 'death_city', 'death_state', 'gender', 'race_or_ethnicity', 'court_type_(1)', 'court_name_(1)', 'appointment_title_(1)', 'appointing_president_(1)', 'party_of_appointing_president_(1)', 'reappointing_president_(1)', 'party_of_reappointing_president_(1)', 'aba_rating_(1)', 'seat_id_(1)', 'statute_authorizing_new_seat_(1)', 'recess_appointment_date_(1)', 'nomination_date_(1)', 'committee_referral_date_(1)', 'hearing_date_(1)', 'judiciary_committee_action_(1)', 'committee_action_date_(1)', 'senate_vote_type_(1)', 'ayes/nays_(1)', 'confirmation_date_(1)', 'commission_date_(1)', 'service_as_chief_judge,_begin_(1)', 'service_as_chief_judge,_end_(1)', '2nd_service_as_chief_judge,_begin_(1)', '2nd_service_as_chief_judge,_end_(1)', 'senior_status_date_(

In [ ]:
# call features.py's normalize_columns function on all DataFrames in dfs, and strip leading and trailing whitespace in all strings
from nomination_predictor.features import normalize_dataframe_columns

for name, df in dfs.items():
    df = normalize_dataframe_columns(df)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    dfs[name] = df

In [ ]:
print("=== Column Names After ===")

for name, df in dfs.items():
    print(f"{name:<35} → {df.columns.tolist()}")

=== Column Names After ===
fjc_judges                          → ['nid', 'jid', 'last_name', 'first_name', 'middle_name', 'suffix', 'birth_month', 'birth_day', 'birth_year', 'birth_city', 'birth_state', 'death_month', 'death_day', 'death_year', 'death_city', 'death_state', 'gender', 'race_or_ethnicity', 'court_type_(1)', 'court_name_(1)', 'appointment_title_(1)', 'appointing_president_(1)', 'party_of_appointing_president_(1)', 'reappointing_president_(1)', 'party_of_reappointing_president_(1)', 'aba_rating_(1)', 'seat_id_(1)', 'statute_authorizing_new_seat_(1)', 'recess_appointment_date_(1)', 'nomination_date_(1)', 'committee_referral_date_(1)', 'hearing_date_(1)', 'judiciary_committee_action_(1)', 'committee_action_date_(1)', 'senate_vote_type_(1)', 'ayes/nays_(1)', 'confirmation_date_(1)', 'commission_date_(1)', 'service_as_chief_judge,_begin_(1)', 'service_as_chief_judge,_end_(1)', '2nd_service_as_chief_judge,_begin_(1)', '2nd_service_as_chief_judge,_end_(1)', 'senior_status_date_(1

### Left-merge nominees table onto nominations table
This will make the cong_noms dataframe we'll use for most of our congress data operations below in this notebook.

In [ ]:
from nomination_predictor.features import merge_nominees_onto_nominations

try:
    # Assuming cong_nominations and cong_nominees dataframes are already loaded
    cong_noms = merge_nominees_onto_nominations(dfs["cong_nominations"], dfs["cong_nominees"])
    
    # Show sample of the merged dataframe
    display(cong_noms.head())
    
    # Report on the merge results
    logger.info(f"Original nominations shape: {cong_nominations.shape}")
    logger.info(f"Original nominees shape: {cong_nominees.shape}")
    logger.info(f"Merged dataframe shape: {cong_noms.shape}")
    
    dfs["cong_noms"] = cong_noms
    
except NameError:
    logger.error("Required dataframes (cong_nominations, cong_nominees) are not defined")
except Exception as e:
    logger.error(f"Error in merge process: {e}")

2025-07-14 17:57:16.459 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:531 - Extracted 5671 URLs from nominees request column (100.0% of rows)
2025-07-14 17:57:16.460 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:555 - Nominations dataframe has 5667 non-null URLs (98.6% of rows)
2025-07-14 17:57:16.475 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:571 - Merged dataframe has 6150 rows
2025-07-14 17:57:16.475 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:572 - Successfully matched 6071 nominations with nominees (98.7%)


,request,retrieval_date,is_full_detail,actions_count,actions_url,authoritydate,citation,committees_count,committees_url,congress,...,nominees_3_ordinal,nominees_3_state,nominees_4_firstname,nominees_4_lastname,nominees_4_ordinal,nominees_4_state,nominees_3_suffix,nominees_1_middlename,nominees_3_middlename,nominees_4_middlename
0,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/118/201...,2025-05-12,PN2013,1.0,https://api.congress.gov/v3/nomination/118/201...,118,...,NaN,None,None,None,NaN,None,None,None,None,None
1,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/118/201...,2025-05-12,PN2012,1.0,https://api.congress.gov/v3/nomination/118/201...,118,...,NaN,None,None,None,NaN,None,None,None,None,None
2,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,11.0,https://api.congress.gov/v3/nomination/118/813...,2025-03-28,PN813,1.0,https://api.congress.gov/v3/nomination/118/813...,118,...,NaN,None,None,None,NaN,None,None,None,None,None
3,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,14.0,https://api.congress.gov/v3/nomination/118/903...,2025-03-28,PN903,1.0,https://api.congress.gov/v3/nomination/118/903...,118,...,NaN,None,None,None,NaN,None,None,None,None,None
4,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,20.0,https://api.congress.gov/v3/nomination/118/816...,2025-03-28,PN816,1.0,https://api.congress.gov/v3/nomination/118/816...,118,...,NaN,None,None,None,NaN,None,None,None,None,None


2025-07-14 17:57:16.487 | INFO     | __main__:<module>:11 - Original nominations shape: (5746, 37)
2025-07-14 17:57:16.487 | INFO     | __main__:<module>:12 - Original nominees shape: (5671, 34)
2025-07-14 17:57:16.488 | INFO     | __main__:<module>:13 - Merged dataframe shape: (6150, 70)


#### Drop duplicated rows (if any) from now-merged congressional datau

Each presidential nomination has one citation.  The first person they nominate would have citation PN1, the thousandth would have PN1000, etc.
This section exists to see (and deal with) we now have multiple rows listing the same citation number and date on which the nomination was received from the President.

In [ ]:
# ------------------------------------------------------------------
# 1. Identify the rows that *would* be dropped
dupe_mask = dfs["cong_noms"].duplicated(subset=["citation", "receiveddate"], keep="first")
dupes      = dfs["cong_noms"].loc[dupe_mask].copy()

# ------------------------------------------------------------------
# 2. Show a compact summary
print(f"Rows flagged as duplicates: {len(dupes)}")
display(
    dupes.sort_values(["citation", "receiveddate"])
         .head(20)   # show first 20; remove .head() to see all
)

# Optional: see how many duplicates per citation
dup_counts = (
    dfs["cong_noms"]
      .loc[dupe_mask, "citation"]
      .value_counts()
      .head(10)
)
print("\nTop duplicate citations:")
display(dup_counts)

Rows flagged as duplicates: 594


,request,retrieval_date,is_full_detail,actions_count,actions_url,authoritydate,citation,committees_count,committees_url,congress,...,nominees_3_ordinal,nominees_3_state,nominees_4_firstname,nominees_4_lastname,nominees_4_ordinal,nominees_4_state,nominees_3_suffix,nominees_1_middlename,nominees_3_middlename,nominees_4_middlename
3685,"{'congress': '106', 'contentType': 'applicatio...",2025-07-12,True,NaN,None,None,PN1-0,NaN,None,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4598,"{'congress': '103', 'contentType': 'applicatio...",2025-07-12,True,NaN,None,None,PN1-0,NaN,None,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4599,"{'congress': '103', 'contentType': 'applicatio...",2025-07-12,True,NaN,None,None,PN1-0,NaN,None,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4600,"{'congress': '103', 'contentType': 'applicatio...",2025-07-12,True,NaN,None,None,PN1-0,NaN,None,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3518,"{'congress': '106', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/106/100...,None,PN1008,1.0,https://api.congress.gov/v3/nomination/106/100...,106,...,NaN,None,None,None,NaN,None,None,None,None,None
3519,"{'congress': '106', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/106/100...,None,PN1008,1.0,https://api.congress.gov/v3/nomination/106/100...,106,...,NaN,None,None,None,NaN,None,None,None,None,None
3520,"{'congress': '106', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/106/100...,None,PN1008,1.0,https://api.congress.gov/v3/nomination/106/100...,106,...,NaN,None,None,None,NaN,None,None,None,None,None
3758,"{'congress': '105', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/105/101...,None,PN1011,1.0,https://api.congress.gov/v3/nomination/105/101...,105,...,NaN,None,None,None,NaN,None,None,None,None,None
3760,"{'congress': '105', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/105/101...,None,PN1011,1.0,https://api.congress.gov/v3/nomination/105/101...,105,...,NaN,None,None,None,NaN,None,None,None,None,None
3761,"{'congress': '105', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/105/101...,None,PN1011,1.0,https://api.congress.gov/v3/nomination/105/101...,105,...,NaN,None,None,None,NaN,None,None,None,None,None



Top duplicate citations:


citation
PN1181-0    63
PN999-0     48
PN1084-0    35
PN1376-0     8
PN864-0      8
PN1901       6
PN1197       6
PN1804       6
PN1070       6
PN1-0        4
Name: count, dtype: int64

In [ ]:
print(f"shape before checking for & dropping duplicated citation items is { dfs["cong_noms"].shape}")
dfs["cong_noms"] = dfs["cong_noms"].drop_duplicates(subset=["citation", "receiveddate"])
print(f"shape after checking for & dropping duplicated citation items is { dfs["cong_noms"].shape}")

shape before checking for & dropping duplicated citation items is (6150, 70)
shape after checking for & dropping duplicated citation items is (5556, 70)


### Drop rows whose congressional citations end in -0
All of these I've seen either:

- lack strictly-necessary information such as nomination & confirmation dates,
- lack helpful information such as the person's name, or 
- whatever little information they do have indicates it's not for a position as a judge (e.g. for secretary of defense, assistant secretary to something-or-other-, etc.) 

In [ ]:
from nomination_predictor.features import filter_dash_zero_citations

dfs["cong_noms"] = filter_dash_zero_citations(dfs["cong_noms"])

2025-07-14 17:57:16.590 | INFO     | nomination_predictor.features:filter_dash_zero_citations:221 - Found 69 citations ending with '-0'
2025-07-14 17:57:16.595 | INFO     | nomination_predictor.features:filter_dash_zero_citations:227 - Removed 69/5556 records with '-0' citations


### Drop non-judge nominations based on position title

In [ ]:
# Filter out non-judicial nominations using the function from features.py
from nomination_predictor.features import filter_non_judicial_nominations

# Define non-judicial titles to filter out
non_judicial_titles = [
    "Attorney", "Board", "Commission", "Director", "Marshal",
    "Assistant", "Representative", "Secretary of", "Member of"
]

dfs["cong_noms"] = filter_non_judicial_nominations(dfs["cong_noms"], non_judicial_titles=non_judicial_titles)

2025-07-14 17:57:16.617 | INFO     | nomination_predictor.features:filter_non_judicial_nominations:187 - Found 1320 unique citations with non-judicial titles
2025-07-14 17:57:16.620 | INFO     | nomination_predictor.features:filter_non_judicial_nominations:193 - Removed 4071/5487 corresponding records


## Few new columns from straightforwardly-parsable data

In [ ]:
from nomination_predictor.name_matching import fill_vacancy_reason_column

dfs["cong_noms"] = fill_vacancy_reason_column(dfs["cong_noms"])

2025-07-14 17:57:16.649 | INFO     | nomination_predictor.name_matching:fill_vacancy_reason_column:404 - Extracted 1416 vacancy reasons from descriptions


In [ ]:
# fill missing values from predecessor column
from nomination_predictor.name_matching import fill_predecessor_column

dfs["cong_noms"] = fill_predecessor_column(dfs["cong_noms"])

In [ ]:
# create a new column in dfs["fjc_federal_judicial_service"] titled "fjc_biography_url"
# whose contents are simply the string "http://www.fjc.gov/node/" concatenated before whatever integer # can be read from 
# that same row's "nid" column (or empty string if nid is not an integer)

svc = dfs["fjc_federal_judicial_service"]

svc["fjc_biography_url"] = (
    "http://www.fjc.gov/node/" +
    pd.to_numeric(svc["nid"], errors="coerce")     # turn non-ints into NaN
      .dropna()                                    # keep only numeric nids
      .astype(int)                                 # cast to int for clean string
      .astype(str)   
)
dfs["fjc_federal_judicial_service"] = svc

## Not yet implemented: Merge only the useful columns of the "other federal judicial service" dataframe

In [ ]:
# left-merges from dfs["fjc_other_federal_judicial_service"] onto dfs["fjc_federal_judicial_service"], but from the former's columns, doesn't left merge all and keep all of them; does the following:
# in the "other service" dataframe, prior to left-merging:
#   drop/ignore all of the columns whose column names stat with "unnamed_"
#   ignore column "judge_name"
#   rename column named "type" to "other_federal_judicial_service_type"
#   rename column named "sequence" to "other_federal_judicial_service_sequence"
#   rename column named "other_federal_judicial_service" to "other_federal_judicial_service_description"

#   adds column named "other_federal_judicial_service_start_date" and "other_federal_judicial_service_end_date" from "other_federal_judicial_service_description"...
# by the time I started looking at how to handle year-parsing, and consider the year-spans in light of the sequence number column in this dataframe, I decided it was higher priority to work on other aspects of this notebook

#### Merge the "Other Nominations" dataframe
In theory this should get us the fjc's perspective on more nominees who didn't get confirmed.  

In practice my runs of name-matching didn't find any unambiguous matches from this dataframe to the congress one, so I put this on hold to try another day, maybe after seeing whether the additional data would help or whether something had to be fixed/improved about the name matcher.

In [ ]:
# left-merges from dfs["fjc_other_nominations_recess"] onto dfs["fjc_federal_judicial_service"]

### Convert date strings to datetime objects

In [ ]:
# for any columns which contain certain keywords in their column name and contain string values, convert from string to datetime
datetime_related_keywords = ("date", "year", "month")

for name, df in dfs.items():
    for col in df.columns:
        if any(keyword in col for keyword in datetime_related_keywords) and df[col].dtype == "object":
            logger.info(f"Converting {col} to datetime for {name}")
            df[col] = pd.to_datetime(df[col], errors="coerce")

2025-07-14 17:57:16.720 | INFO     | __main__:<module>:7 - Converting birth_year to datetime for fjc_judges
2025-07-14 17:57:16.725 | INFO     | __main__:<module>:7 - Converting recess_appointment_date_(1) to datetime for fjc_judges
2025-07-14 17:57:16.728 | INFO     | __main__:<module>:7 - Converting nomination_date_(1) to datetime for fjc_judges
2025-07-14 17:57:16.731 | INFO     | __main__:<module>:7 - Converting committee_referral_date_(1) to datetime for fjc_judges
2025-07-14 17:57:16.733 | INFO     | __main__:<module>:7 - Converting hearing_date_(1) to datetime for fjc_judges
2025-07-14 17:57:16.738 | INFO     | __main__:<module>:7 - Converting committee_action_date_(1) to datetime for fjc_judges
2025-07-14 17:57:16.742 | INFO     | __main__:<module>:7 - Converting confirmation_date_(1) to datetime for fjc_judges
2025-07-14 17:57:16.744 | INFO     | __main__:<module>:7 - Converting commission_date_(1) to datetime for fjc_judges
2025-07-14 17:57:16.748 | INFO     | __main__:<modul

### Normalize all string values to make matching them later easier

In [ ]:
from nomination_predictor.name_matching import normalize_text

keywords_which_denote_string_columns_to_normalize = ("court", "circuit", "district", "description", "name")

for name, df in dfs.items():
    for col in df.columns:
        if any(keyword in col.casefold() for keyword in keywords_which_denote_string_columns_to_normalize) and df[col].dtype == object:
            logger.info(F"Normalizing all values within column named {col} in {name}")
            df[col] = df[col].apply(normalize_text)

2025-07-14 17:57:17.016 | INFO     | __main__:<module>:8 - Normalizing all values within column named last_name in fjc_judges
2025-07-14 17:57:17.025 | INFO     | __main__:<module>:8 - Normalizing all values within column named first_name in fjc_judges
2025-07-14 17:57:17.032 | INFO     | __main__:<module>:8 - Normalizing all values within column named middle_name in fjc_judges
2025-07-14 17:57:17.042 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_type_(1) in fjc_judges
2025-07-14 17:57:17.052 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_name_(1) in fjc_judges
2025-07-14 17:57:17.061 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_type_(2) in fjc_judges
2025-07-14 17:57:17.065 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_name_(2) in fjc_judges
2025-07-14 17:57:17.069 | INFO     | __main__:<module>:8 - Normalizing all values within column

### Count and display unique values under each column

In [ ]:
# display counts of unique values in DataFrame columns:
for name, df in dfs.items():
    for col in sorted(df.columns):
     print(f"{name} - {col}: {df[col].nunique()} unique values")

fjc_judges - 2nd_service_as_chief_judge,_begin_(1): 5 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(2): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(3): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(4): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(5): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(6): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(1): 4 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(2): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(3): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(4): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(5): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(6): 0 unique values
fjc_judges - aba_rating_(1): 6 unique values
fjc_judges - aba_rating_(2): 5 unique values
fjc_judges - aba_rating_(3): 3 unique values
fjc_judges - aba_rating_(4): 1 unique values
fjc_judges - aba

## Name-matching FJC judges to Congress.gov nominees

### For confirmed judges

#### Supplementing with additional columns to aid matching

In [ ]:
# add a "full_name_concatenated" column to the fjc_federal_judicial_service dataframe which is composed by flipping its judge_name column values 
# from "lastname, firstname middleNameOrMiddleInitial (, optional comma and suffix)" to "firstname lastname middle suffix"
from nomination_predictor.features import \
    convert_judge_name_format_from_last_comma_first_to_first_then_last

try:
    dfs["fjc_federal_judicial_service"]["full_name_concatenated"] = dfs["fjc_federal_judicial_service"]["judge_name"].apply(convert_judge_name_format_from_last_comma_first_to_first_then_last)
    
    # Show some examples to verify the conversion
    sample = dfs["fjc_federal_judicial_service"][['judge_name', 'full_name_concatenated']].head(10)
    display(sample)
    
    # Count null values to check for any conversion failures
    null_count = dfs["fjc_federal_judicial_service"]["full_name_concatenated"].isna().sum()
    empty_count = (dfs["fjc_federal_judicial_service"]["full_name_concatenated"] == '').sum()
    
    if null_count > 0 or empty_count > 0:
        logger.warning(f"Found {null_count} null values and {empty_count} empty strings in the converted names.")
        
    logger.info(f"Successfully added 'full_name_concatenated' column to fjc_federal_judicial_service dataframe with {len(dfs["fjc_federal_judicial_service"])}) entries.")
    
except Exception as e:
    logger.error(f"Error creating full_name_concatenated column: {e}")
    # If there's an error, display the first few rows of fjc_federal_judicial_service to help diagnose
    logger.info("\nSample of fjc_federal_judicial_service dataframe:")
    display(dfs["fjc_federal_judicial_service"].head(3))
    logger.info(f"Columns available: {dfs["fjc_federal_judicial_service"].columns.tolist()}")

,judge_name,full_name_concatenated
0,"abelson, adam ben",adam ben abelson
1,"abrams, ronnie",ronnie abrams
2,"abruzzo, matthew t.",matthew t. abruzzo
3,"abudu, nancy gbana",nancy gbana abudu
4,"acheson, marcus wilson",marcus wilson acheson
5,"acheson, marcus wilson",marcus wilson acheson
6,"acheson, marcus wilson",marcus wilson acheson
7,"acker, william marsh, jr.",william marsh acker jr.
8,"ackerman, harold arnold",harold arnold ackerman
9,"ackerman, james waldo",james waldo ackerman


2025-07-14 17:57:17.667 | INFO     | __main__:<module>:20 - Successfully added 'full_name_concatenated' column to fjc_federal_judicial_service dataframe with 4720) entries.


In [ ]:
# add a "full_name_from_description" and a "location_of_origin_from_description" columns to the dfs["cong_noms"] dataframe which regex-captures the first segments of the same dfs["cong_noms"] dataframe row's "description" string, 
# i.e. captures name before the first appearances of the phrases ", of " or ", of the "
# and captures location from the second segment of the same dfs["cong_noms"] dataframe row's "description" string
# i.e. captures between the above-seen phrase ", of " or ", of the " through to the phrase ", to be "
# examples: 
# melissa damian, of florida, to be ...  gets captured into those new columns as "melissa damian" and "florida"
# nicole g. bernerr of maryland, to be united... gets captured into those new columns as "nicole g. bernerr" and "maryland"
# kirk edward sherriff, of california, to be united... gets captured into those new columns as "kirk edward sherriff" and "california"
# sherri malloy beatty-arthur, of the district of columbia, for... gets captured into those new columns as "sherri malloy beatty-arthur" and "district of columbia"

# Extract full_name_from_description and location_of_origin_from_description from description field
from nomination_predictor.features import extract_name_and_location_columns

# Apply the extraction function to cong_noms dataframe
if 'cong_noms' in dfs:
    dfs['cong_noms'] = extract_name_and_location_columns(dfs['cong_noms'])
    
    # Display sample results to verify extraction
    sample_cols = ['description', 'full_name_from_description', 'location_of_origin_from_description']
    display(dfs['cong_noms'][sample_cols].head(10))
    
    # Report extraction statistics
    total_rows = len(dfs['cong_noms'])
    name_filled = dfs['cong_noms']['full_name_from_description'].notna().sum()
    location_filled = dfs['cong_noms']['location_of_origin_from_description'].notna().sum()
    
    logger.info(f"Extracted names for {name_filled}/{total_rows} records ({name_filled/total_rows:.1%})")
    logger.info(f"Extracted locations for {location_filled}/{total_rows} records ({location_filled/total_rows:.1%})")
else:
    logger.error("Error: 'cong_noms' dataframe not found in dfs dictionary.")

2025-07-14 17:57:17.692 | INFO     | nomination_predictor.features:extract_name_and_location_columns:700 - Extracted 1416/1416 (100.0%) names and 1416/1416 (100.0%) locations


,description,full_name_from_description,location_of_origin_from_description
0,"nicholas george miranda, of the district of co...",nicholas george miranda,district of columbia
1,"james graham lake, of the district of columbia...",james graham lake,district of columbia
16,"mustafa taher kasubhai, of oregon, to be unite...",mustafa taher kasubhai,oregon
21,"jacqueline becerra, of florida, to be united s...",jacqueline becerra,florida
24,"jacquelyn d. austin, of south carolina, to be ...",jacquelyn d. austin,south carolina
31,"melissa damian, of florida, to be united state...",melissa damian,florida
36,"nicole g. berner, of maryland, to be united st...",nicole g. berner,maryland
43,"kirk edward sherriff, of california, to be uni...",kirk edward sherriff,california
48,"sherri malloy beatty-arthur, of the district o...",sherri malloy beatty-arthur,district of columbia
49,"erin camille johnston, of the district of colu...",erin camille johnston,district of columbia


2025-07-14 17:57:17.701 | INFO     | __main__:<module>:27 - Extracted names for 1416/1416 records (100.0%)
2025-07-14 17:57:17.702 | INFO     | __main__:<module>:28 - Extracted locations for 1416/1416 records (100.0%)


#### Performing the matching operations

In [ ]:
from nomination_predictor.name_matching import perform_exact_name_matching

results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices= perform_exact_name_matching(
    congress_df=dfs["cong_noms"],
    fjc_df=dfs["fjc_federal_judicial_service"],
    congress_name_col="full_name_from_description",
    fjc_name_col="judge_name"
)

2025-07-14 17:57:17.713 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:123 - Starting exact name matching with 1416 Congress records and 4720 FJC records
2025-07-14 17:57:17.936 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:142 - Performing first-pass join on last and first name
2025-07-14 17:57:17.949 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:149 - Found 954 total records with last+first name matches
2025-07-14 17:57:17.953 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:177 - Found 22 ambiguous matches, attempting middle initial disambiguation
2025-07-14 17:57:17.954 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:181 - Samples of pending ambiguous rows
2025-07-14 17:57:17.955 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:182 -                                                 request retrieval_date  \
384   {

In [ ]:
# Show results
results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices.head()

,congress_index,congress_name,fjc_name,nid,match_type,ambiguous
0,16,mustafa taher kasubhai,"kasubhai, mustafa taher",13761892.0,first_and_last_name,False
1,21,jacqueline becerra,"becerra, jacqueline",13761516.0,first_and_last_name,False
2,24,jacquelyn d. austin,"austin, jacquelyn denise",13761451.0,first_and_last_name,False
3,31,melissa damian,"damian, melissa",13761531.0,first_and_last_name,False
4,36,nicole g. berner,"berner, nicole gina",13761561.0,first_and_last_name,False


In [ ]:
# keep only *unambiguous* pairs
nid_map = (
    results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices[~results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices["ambiguous"]]        # drop rows still ambiguous
      .set_index("congress_index")["nid"]
)

In [ ]:
# at long last, we have a way to bridge the gap between the congress.gov data and the fjc data

# we can now use the nid_map to add the nid column to the congress.gov data
dfs["cong_noms"]["nid"] = dfs["cong_noms"].index.to_series().map(nid_map)
cong_noms = dfs["cong_noms"]

### For unconfirmed judges

In practice, given the dataframes as I've got them as of typing this, this section doesn't find any remaining unconfirmed judges to match.

What this section _did_ accomplish was showing me that the presence of diacritical marks such as "ñ" or "é" in names was misleading the matching process.

Discovering and addressing that in much-earlier data-normalizing cells led to getting more matches in our confirmed-judges-matching notebook section.

#### Supplementing with additional columns to aid matching

In [ ]:
from nomination_predictor.name_matching import prep_fjc_other

dfs["fjc_other_nominations_recess"] = prep_fjc_other(fjc_other_df=dfs["fjc_other_nominations_recess"])

In [ ]:
fjc_other_supplemented =dfs["fjc_other_nominations_recess"]

#### Performing the matching operation

In [ ]:
from nomination_predictor.features import link_unconfirmed_nominations

dfs["cong_noms"] = link_unconfirmed_nominations(dfs["cong_noms"], dfs["fjc_other_nominations_recess"])

2025-07-14 17:57:18.199 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:123 - Starting exact name matching with 248 Congress records and 828 FJC records
2025-07-14 17:57:18.245 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:142 - Performing first-pass join on last and first name
2025-07-14 17:57:18.252 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:149 - Found 0 total records with last+first name matches
2025-07-14 17:57:18.253 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:153 - NO last+first name matches found. Checking last-name-only matches for diagnosis...
2025-07-14 17:57:18.256 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:157 - Found 75 last-name-only matches
2025-07-14 17:57:18.256 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:159 - Showing up to first 10 last-name-only matches:
2025-07-14 17:57:18.257 | 

## Combining the rest of the FJC data now that our congress dataframe has been enriched with FJC nid

### Handling nominees' education and job history

Before we combine FJC data, we have to consider whether/how to handle judges' education, job history, age, ABA rating, etc.  Most/all of the data in the "demographics" dataframe is unchanging over time, but that's very much _not_ true of the other dataframes.

The simplest way to handle it would be to left-merge on "nid" and only take the most recently-dated row, or row with the highest sequence number.  In most cases this would likely land on keeping the most prestigious degree or job.

However, it is entirely likely a judge's education or job history has changed substantially since their first nomination, and affected their qualifications for each later nomination.

All of these indicate to me that it's worth merging onto each row that judge's position, education, etc., not as of the most recent records available, but instead _as of when they were nominated._

That means we can't do a too-simple left-join of all of our FJC data.  Instead, now that we've done the step of matching NIDs to congress' data on nominations, we can use the "received date" for each congress citation as a cutoff date for when we lookup education and job records by "nid" -- so we can avoid mistakenly linking to a citation any employment & job records dated after that cutoff date.

Thankfully we do have the school, degree, and degree_year in the education record, for both their bachelors and their masters and their associate degree(s) and LLB and J.D. etc., so we can look that up.  The education dataframe even comes with a "sequence" number for each education record, which is another indicator of chronological order in addition to degree_year for any given "nid" lookup for a judge.

Job history is more challenging to deal with because literally every row entry in that dataframe lists it uniquely, but we do have the data available.  My earliest attempts to feature-engineer with it include looking for keywords in it, then creating boolean features for whether they did/didn't have experience in common-phrase-identifiable positions such as "Private practice" or "Attorney general" or "Navy" or "Army" etc. Theoretically a parser can look for the year spreads listed there as a rough indicator of amounts of experience gleaned from each professional role & when, but that may be too complicated for me to accomplish by the time I'm first presenting this work.

In [ ]:
import pandas as pd

from nomination_predictor.time_aware_analysis import merge_congress_fjc

# Perform the time-aware merge
time_aware_merged_df = merge_congress_fjc(dfs["cong_noms"], dfs["fjc_judges"], dfs["fjc_demographics"],
                                                  dfs["fjc_education"], dfs["fjc_professional_career"], dfs["fjc_federal_judicial_service"])
display(time_aware_merged_df.head(5))

/home/wsl2ubuntuuser/nomination_predictor/nomination_predictor/time_aware_analysis.py:256: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged[col] = merged[col].fillna(False)


,request,retrieval_date,is_full_detail,actions_count,actions_url,authoritydate,citation,committees_count,committees_url,congress,...,degree_(4),degree_year_(4),school_(5),degree_(5),degree_year_(5),professional_career,other_nominations/recess_appointments,jid_fjcdemo,gender_fjcdemo,race_or_ethnicity_fjcdemo
0,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/118/201...,2025-05-12,PN2013,1.0,https://api.congress.gov/v3/nomination/118/201...,118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,6.0,https://api.congress.gov/v3/nomination/118/201...,2025-05-12,PN2012,1.0,https://api.congress.gov/v3/nomination/118/201...,118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,11.0,https://api.congress.gov/v3/nomination/118/102...,2025-03-28,PN1024,1.0,https://api.congress.gov/v3/nomination/118/102...,118,...,NaN,NaN,NaN,NaN,NaN,"Landlord-tenant mediation coordinator, Lane Co...",Nominated to U.S. District Court for the Distr...,13761892.0,Male,Asian American
3,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,3.0,https://api.congress.gov/v3/nomination/118/113...,2025-03-28,PN1131,1.0,https://api.congress.gov/v3/nomination/118/113...,118,...,NaN,NaN,NaN,NaN,NaN,"Trial attorney, Federal Programs Branch, Civil...",Nominated to U.S. District Court for the South...,13761516.0,Female,Hispanic
4,"{'congress': '118', 'contentType': 'applicatio...",2025-07-12,True,12.0,https://api.congress.gov/v3/nomination/118/113...,2025-03-28,PN1130,1.0,https://api.congress.gov/v3/nomination/118/113...,118,...,NaN,NaN,NaN,NaN,NaN,"Law clerk, Hon. Matthew J. Perry, Jr., U.S. Di...",NaN,13761451.0,Female,African American


### Feature engineering 

In [ ]:
from datetime import date

from nomination_predictor.time_aware_analysis import (
    congress_number, congress_session, days_into_current_term,
    days_until_next_midterm_election, days_until_next_presidential_election,
    fill_missing_appointing_presidents, president_number,
    presidential_term_index)

df = time_aware_merged_df.copy()
df["receiveddate"] = pd.to_datetime(df["receiveddate"])   # ensure datetime

In [ ]:
# presidency- and elections-timeline-related
df["pres_term_idx"]  = df["receiveddate"].apply(presidential_term_index)
df["days_into_pres_term"] = df["receiveddate"].apply(days_into_current_term)
df["days_to_next_pres_election"] = df["receiveddate"].apply(days_until_next_presidential_election)
df["days_to_next_midterm_election"]  = df["receiveddate"].apply(days_until_next_midterm_election)
df["congress_num"] = df["receiveddate"].apply(congress_number)
df["congress_session"] = df["receiveddate"].apply(congress_session)

In [ ]:
df = fill_missing_appointing_presidents(df)

2025-07-14 17:57:19.298 | INFO     | nomination_predictor.time_aware_analysis:fill_missing_appointing_presidents:382 - Filled 489 missing appointing president values using nomination dates


Among the hypotheses is that older judges tend to get approved faster because there's not as much concern they'll live long enough to have as much of a total impact over their time in office.

In [ ]:
# Age at nomination
from datetime import datetime, timedelta

# Reference date for “future” check
yesterday = (pd.Timestamp.today().normalize() - pd.Timedelta(days=1))

#  Mask rows that have *all three* components
complete_date_mask = (
    df['birth_day'].notna()  &
    df['birth_month'].notna() &
    df['birth_year'].notna()
)

#  Initialise column
df['birth_date'] = pd.NaT

#  Build datetime only for rows that passed the mask
tmp_birth = pd.to_datetime(
    {
        'year':  df.loc[complete_date_mask, 'birth_year'],
        'month': df.loc[complete_date_mask, 'birth_month'],
        'day':   df.loc[complete_date_mask, 'birth_day']
    },
    errors='coerce'           # invalid combos → NaT
)

# Keep the value only if they were at least "born yesterday, i.e. ignore misfiled future dates
tmp_birth = tmp_birth.where(tmp_birth < yesterday, pd.NaT)

df.loc[complete_date_mask, 'birth_date'] = tmp_birth

df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')

df['age_at_nom'] = (
    (df['receiveddate'] - df['birth_date']).dt.days // 365.25
).astype('Int64')

In [ ]:
# identify seat level
df["seat_level"] = (
    df["description"] # we do have court_type_(1), # court_type_(2), etc. columns, but for this coarse an analysis it's simpler and accurate-enough to look through the nomination description
      .str.lower()
      .str.extract(r"(supreme|circuit|district|(?<=\s)tax|international|appeals)") # insists on whitespace before "tax" so we can tell "tax" or "taxation" etc. are a standalone word
      .fillna("other")
)

### Not-yet-implemented analyses ideas:

In [ ]:
# Partisan mismatch: 1 if president_party != party__who_appointed_predecessor
# this would require an additional step of linking predecessor name to nid, and way of looking up when the judge with that nid had been in service, possibly even needing to be a date-and-location-aware analysis

#party_map = {47: "R", 46: "D", 45: "R", 44: "D", 43: "R", 42: "D", 41: "R"}  # extend list
#df["pres_party"] = df["receiveddate"].apply(lambda d: party_map.get(president_number(d), None))
#df["partisan_mismatch"] = (
#    (df["pres_party"].notna()) &
#    (df["pres_party"] != df["party_of_appointing_president"])
#)

In [ ]:

# skipping this one because I think we'd get more and/or richer info out of it if we had a more-successful linkage of the unconfirmed nomination rows between congress and fjc's data
# #Count prior failed nominations for this seat_id (if column present)

#if "other_nominations_count" not in df.columns and "seat_id" in df.columns:
#    prior_counts = (
#        df.groupby("seat_id").cumcount()  # number seen so far for that seat
#    )
#    df["num_prior_failed_noms"] = prior_counts
#
#display(df.head())
#feature_engineered_df = df.copy()

## Saving interim dataframes

In [ ]:
# Save to interim data
df.to_csv( INTERIM_DATA_DIR /"feature_engineered.csv", index=False)

In [ ]:
# Save extracted tables to interim directory
for name, df in dfs.items():
    if len(df) > 0:  # Only save non-empty DataFrames
        output_path = INTERIM_DATA_DIR / f"{name}.csv"
        df.to_csv(output_path, index=False)
        print(f"Saved {len(df)} records to {output_path}")

Saved 4022 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_judges.csv
Saved 4720 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_federal_judicial_service.csv
Saved 4022 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_demographics.csv
Saved 8040 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_education.csv
Saved 611 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_other_federal_judicial_service.csv
Saved 828 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_other_nominations_recess.csv
Saved 19003 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_professional_career.csv
Saved 5746 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/cong_nominations.csv
Saved 5671 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/cong_nominees.csv
Saved 1416 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/cong_